In [27]:
#!/usr/bin/env python
# coding: utf-8

# In[4]:


#!/usr/bin/env python
# coding: utf-8
#手指スケルトンの可視化プログラム(3次元XYZデータ用)
import glob
import os
import csv
import copy
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

sns.set_style("darkgrid")
INPUT_AREA = False #グラフ軸範囲をCSVで指定する場合True
OBJECT_BASE = True #物体基準座標スケルトンの場合True、手首位置基準座標スケルトンの場合False
ZERO_FIX = True    #値0の点とのエッジの非表示化を行う場合True
CONFIDENCE = False #信頼度値が付属しているデータの場合True

#os.chdir("./CSV_Visualizer_python")
#os.chdir("../")
#実行ディレクトリを取得
dir_csv_viz = os.getcwd()
#入力ディレクトリ名を指定
dir_input = dir_csv_viz + "\\input"
#出力ディレクトリ名を指定
dir_output =dir_csv_viz + "\\output"

#print(os.getcwd())
print("Current Directory is : " + dir_csv_viz)
print("Input Directory is : " + dir_input)
print("Output Directory is : " + dir_output)

#取得するCSVファイルの名前規則設定ファイルを読み込み
fp_ftype_conf = open(dir_csv_viz + "\\configFileType.txt", "r")
file_type = ""
for line in fp_ftype_conf:
    file_type = line
fp_ftype_conf.close()
print(file_type)

#入力ディレクトリ内CSVファイルの名前リストを取得
csv_name_list = glob.glob(dir_input + "\\" + file_type)
#print(csv_name_list)

#プロット軸の値範囲を固定するために、全CSVを走査して、x,y,zそれぞれの値の最大・最小を調べる
x_min = 0
x_max = 0
y_min = 0
y_max = 0
for s in csv_name_list:
    fp_search = open(s, "r")
    searcher = csv.reader(fp_search)
    data = [row for row in searcher]
    #print(data[0])
    points = [float(t) for t in data[0]]
    #print(points)
    #print(len(points))
    x_values = []
    y_values = []
    if CONFIDENCE:
        for i in range(int(len(points)/3)):
            x_values.append(points[i*3])
            y_values.append(-1*points[i*3+1])
    else:
        for i in range(int(len(points)/2)):
            if points[i*2] < -5000 and points[i*2] != -10000:
                print("too high value error X", s)
            if points[i*2+1] < -5000 and points[i*2+1] != -10000:
                print("too high value error Y", s)
            x_values.append(points[i*2])
            y_values.append(-1*points[i*2+1])
    #外れ値になっている要素(-10000)を削除
    x_values = [out for out in x_values if out > -5000]
    y_values = [out for out in y_values if out < 5000]
    #print(x_values)
    #print(y_values)
    
    tx_max = max(x_values, default = -10000)
    tx_min = min(x_values, default = 10000)
    ty_max = max(y_values, default = -10000)
    ty_min = min(y_values, default = 10000)
    if x_min == 0 and x_max == 0 and y_min == 0 and y_max == 0:
        x_min = tx_min
        x_max = tx_max
        y_min = ty_min
        y_max = ty_max
    if tx_max > x_max:
        x_max = tx_max
    if tx_min < x_min:
        x_min = tx_min
    if ty_max > y_max:
        y_max = ty_max
    if ty_min < y_min:
        y_min = ty_min
    fp_search.close()
    
print(x_min, x_max, y_min, y_max, "(x_min, x_max, y_min, y_max)")

fp_area_out = open(dir_output + "\\area.csv", "w")
awriter = csv.writer(fp_area_out)
awriter.writerow([x_min, x_max, y_min, y_max])
fp_area_out.close()

X_MIN_LIM = -1150
X_MAX_LIM = 920
Y_MIN_LIM = -1440
Y_MAX_LIM = 1410
Z_MIN_LIM = -1690
Z_MAX_LIM = 2160

HAND_PNT_NUM = 21

if INPUT_AREA:
    fp_area = open(dir_csv_viz + "\\area.csv", "r")
    areader = csv.reader(fp_area)
    area_data = [row for row in areader]
    areas = [float(a) for a in area_data[0]]
    fp_area.close()
    x_min = areas[0]
    x_max = areas[1]
    y_min = areas[2]
    y_max = areas[3]
else:
    if x_min < X_MIN_LIM:
        x_min = X_MIN_LIM
    if x_max > X_MAX_LIM:
        x_max = X_MAX_LIM
    if y_min < Y_MIN_LIM:
        y_min = Y_MIN_LIM
    if y_max > Y_MAX_LIM:
        y_max = Y_MAX_LIM

print(x_min, x_max, y_min, y_max, "(fixed: x_min, x_max, y_min, y_max)")
    
csv_data_list = glob.glob(dir_input + "\\" + file_type)

#CSVファイル名リストのファイルを1つずつ可視化していく
for f in csv_data_list:
    fig = plt.figure()
    fp_csv = open(f, "r")
    
    #1つのCSVからデータを読み取る
    reader = csv.reader(fp_csv)
    data = [row for row in reader]    
    points = [float(t) for t in data[0]]
    #print(points)
    #print(len(points))
    #x,y,zそれぞれの値列として取得
    x_values = []
    y_values = []
    if CONFIDENCE:
        for i in range(int(len(points)/3)):
            x_values.append(points[i*3])
            y_values.append(-1*points[i*3+1])
    else:
        for i in range(int(len(points)/2)):
            x_values.append(points[i*2])
            y_values.append(-1*points[i*2+1])
    #print(ids)
    #print(x_values)
    #print(y_values)
    #print(confidences)
    #print(z_values)
    
    #x,y,zをプロット
    #ax = Axes3D(fig)
    ax = fig.add_subplot(111)
    ax.set_xlim([x_min, x_max])
    ax.set_ylim([y_min, y_max])
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    #ax.plot(x_values, z_values, y_values, marker = "o", linestyle = 'None')
    #plot(1,5, ms=20., marker='.', label='point')
    #各点をプロット
    ax.scatter(x_values[0], y_values[0], s=10, c='#e53528', marker = ".")
    ax.scatter(x_values[1], y_values[1], s=10, c='#e77340', marker = ".")
    ax.scatter(x_values[2], y_values[2], s=10, c='#eb8e25', marker = ".")
    ax.scatter(x_values[3], y_values[3], s=10, c='#d7a10e', marker = ".")
    ax.scatter(x_values[4], y_values[4], s=10, c='#d9c812', marker = ".")
    ax.scatter(x_values[5], y_values[5], s=10, c='#a7cf21', marker = ".")
    ax.scatter(x_values[6], y_values[6], s=10, c='#7fe545', marker = ".")
    ax.scatter(x_values[7], y_values[7], s=10, c='#53d62d', marker = ".")
    ax.scatter(x_values[8], y_values[8], s=10, c='#4abc2f', marker = ".")
    ax.scatter(x_values[9], y_values[9], s=10, c='#52d686', marker = ".")
    ax.scatter(x_values[10], y_values[10], s=10, c='#52dfbe', marker = ".")
    ax.scatter(x_values[11], y_values[11], s=10, c='#52b9c5', marker = ".")
    ax.scatter(x_values[12], y_values[12], s=10, c='#508ab7', marker = ".")
    ax.scatter(x_values[13], y_values[13], s=10, c='#556fd0', marker = ".")
    ax.scatter(x_values[14], y_values[14], s=10, c='#5245d5', marker = ".")
    ax.scatter(x_values[15], y_values[15], s=10, c='#7e2fba', marker = ".")
    ax.scatter(x_values[16], y_values[16], s=10, c='#bc64fb', marker = ".")
    ax.scatter(x_values[17], y_values[17], s=10, c='#da53e8', marker = ".")
    ax.scatter(x_values[18], y_values[18], s=10, c='#eb45bc', marker = ".")
    ax.scatter(x_values[19], y_values[19], s=10, c='#ec3c89', marker = ".")
    ax.scatter(x_values[20], y_values[20], s=10, c='#ff6691', marker = ".")
    #ax.plot(x_values[0], z_values[0], y_values[0], ms = 20, marker = ".")
    
    #点が描画範囲内かどうか
    isPointsIn = [False] * HAND_PNT_NUM
    #print(isPointsIn)
    
    #すべての点について
    #指定エリア内に含まれているか　いるならTrue
    for p in range(HAND_PNT_NUM):
        if x_min <= x_values[p] <= x_max and y_min <= y_values[p] <= y_max:
            isPointsIn[p] = True
    #値0の点とのエッジを非表示にするか　するならTrue
    if ZERO_FIX == True:
        #物体基準座標系での可視化の場合
        if OBJECT_BASE == True:
            for p in range(HAND_PNT_NUM):
                if x_values[p] == 0 and y_values[p] == 0:
                    isPointsIn[p] = False
        #手首位置基準座標系での可視化の場合(手首位置が[0,0,0]なのでそことのエッジだけは非表示にしない)
        elif OBJECT_BASE == False:
            for p in range(HAND_PNT_NUM):
                if p != 0 and x_values[p] == 0 and y_values[p] == 0:
                    isPointsIn[p] = False
    
    #各エッジのプロット
    if isPointsIn[0] and isPointsIn[1]:
        ax.plot([x_values[0],x_values[1]], [y_values[0],y_values[1]], c='#cc0000')
    if isPointsIn[1] and isPointsIn[2]:
        ax.plot([x_values[1],x_values[2]], [y_values[1],y_values[2]], c='#b30000')
    if isPointsIn[2] and isPointsIn[3]:
        ax.plot([x_values[2],x_values[3]], [y_values[2],y_values[3]], c='#e60000')
    if isPointsIn[3] and isPointsIn[4]:
        ax.plot([x_values[3],x_values[4]], [y_values[3],y_values[4]], c='#ff0000')
    if isPointsIn[0] and isPointsIn[5]:
        ax.plot([x_values[0],x_values[5]], [y_values[0],y_values[5]], c='#a3cc00')
    if isPointsIn[5] and isPointsIn[6]:
        ax.plot([x_values[5],x_values[6]], [y_values[5],y_values[6]], c='#8fb300')
    if isPointsIn[6] and isPointsIn[7]:        
        ax.plot([x_values[6],x_values[7]], [y_values[6],y_values[7]], c='#b8e600')
    if isPointsIn[7] and isPointsIn[8]:
        ax.plot([x_values[7],x_values[8]], [y_values[7],y_values[8]], c='#ccff00')
    if isPointsIn[0] and isPointsIn[9]:
        ax.plot([x_values[0],x_values[9]], [y_values[0],y_values[9]], c='#00cc52')
    if isPointsIn[9] and isPointsIn[10]:
        ax.plot([x_values[9],x_values[10]], [y_values[9],y_values[10]], c='#00b347')
    if isPointsIn[10] and isPointsIn[11]:
        ax.plot([x_values[10],x_values[11]], [y_values[10],y_values[11]], c='#00e65c')
    if isPointsIn[11] and isPointsIn[12]:
        ax.plot([x_values[11],x_values[12]], [y_values[11],y_values[12]], c='#00ff66')
    if isPointsIn[0] and isPointsIn[13]:
        ax.plot([x_values[0],x_values[13]], [y_values[0],y_values[13]], c='#0052cc')
    if isPointsIn[13] and isPointsIn[14]:
        ax.plot([x_values[13],x_values[14]], [y_values[13],y_values[14]], c='#0047b3')
    if isPointsIn[14] and isPointsIn[15]:
        ax.plot([x_values[14],x_values[15]], [y_values[14],y_values[15]], c='#005ce6')
    if isPointsIn[15] and isPointsIn[16]:
        ax.plot([x_values[15],x_values[16]], [y_values[15],y_values[16]], c='#0066ff')
    if isPointsIn[0] and isPointsIn[17]:
        ax.plot([x_values[0],x_values[17]], [y_values[0],y_values[17]], c='#a300cc')
    if isPointsIn[17] and isPointsIn[18]:
        ax.plot([x_values[17],x_values[18]], [y_values[17],y_values[18]], c='#8f00b3')
    if isPointsIn[18] and isPointsIn[19]:
        ax.plot([x_values[18],x_values[19]], [y_values[18],y_values[19]], c='#b800e6')
    if isPointsIn[19] and isPointsIn[20]:
        ax.plot([x_values[19],x_values[20]], [y_values[19],y_values[20]], c='#cc00ff')
    
    if False:
        fp_obox_csv = open(dir_csv_viz+"\\objectbox.csv", "r")
        objreader = csv.reader(fp_obox_csv)
        objdata = [row for row in objreader]
        objpoints = [float(t) for t in objdata[0]]
        o1x = objpoints[0]
        o1y = (-1*objpoints[1])
        o2x = objpoints[2]
        o2y = (-1*objpoints[3])
        o3x = objpoints[4]
        o3y = (-1*objpoints[5])
        o4x = objpoints[6]
        o4y = (-1*objpoints[7])
        #print(o1x, o1y, o2x, o2y, o3x, o3y, o4x, o4y)
        xmid = (o1x+o2x)/2
        ymid = (o1y+o3y)/2
        o1x -= xmid
        o2x -= xmid
        o3x -= xmid
        o4x -= xmid
        o1y -= ymid
        o2y -= ymid
        o3y -= ymid
        o4y -= ymid
        ax.scatter(o1x, o1y, s=10, c='#0000FF', marker = ".")
        ax.scatter(o2x, o2y, s=10, c='#0000FF', marker = ".")
        ax.scatter(o3x, o3y, s=10, c='#0000FF', marker = ".")
        ax.scatter(o4x, o4y, s=10, c='#0000FF', marker = ".")
        ax.plot([o1x,o2x], [o1y,o2y], c='#0000FF')
        ax.plot([o2x,o4x], [o2y,o4y], c='#0000FF')
        ax.plot([o4x,o3x], [o4y,o3y], c='#0000FF')
        ax.plot([o3x,o1x], [o3y,o1y], c='#0000FF')
        fp_obox_csv.close()
    ax.set
    #plt.show()
    
    #出力画像ファイルパスを指定
    
    #ファイル名の生成
    fname = str(copy.copy(f))
    fname = fname.split(dir_input)[1]
    #ax.set_title(fname)
    fname = fname.replace(".csv", ".png")
    #print("fname: " + fname)
    
    #保存ファイルパスの生成
    fpath = dir_output + "\\" + fname
    #print(fpath)
    
    plt.savefig(fpath, dpi = 300,transparent = False, bbox_inches = 'tight', pad_inches = 0)
    fp_csv.close()
    plt.close()
print("completed saving !")

Current Directory is : C:\Users\administrator\CSV_Visualizer_python
Input Directory is : C:\Users\administrator\CSV_Visualizer_python\input
Output Directory is : C:\Users\administrator\CSV_Visualizer_python\output
*.csv
-21.710945 203.91502 -25.907347 154.63412 (x_min, x_max, y_min, y_max)
-21.710945 203.91502 -25.907347 154.63412 (fixed: x_min, x_max, y_min, y_max)
completed saving !
